In [2]:
import pandas as pd
df = pd.read_csv(r"F:\Data science\train.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550068 entries, 0 to 550067
Data columns (total 12 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   User_ID                     550068 non-null  int64  
 1   Product_ID                  550068 non-null  object 
 2   Gender                      550068 non-null  object 
 3   Age                         550068 non-null  object 
 4   Occupation                  550068 non-null  int64  
 5   City_Category               550068 non-null  object 
 6   Stay_In_Current_City_Years  550068 non-null  object 
 7   Marital_Status              550068 non-null  int64  
 8   Product_Category_1          550068 non-null  int64  
 9   Product_Category_2          376430 non-null  float64
 10  Product_Category_3          166821 non-null  float64
 11  Purchase                    550068 non-null  int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 50.4+ MB


In [3]:
selected_df = df[['Gender', 'Age', 'Occupation', 'City_Category', 'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category_1', 'Purchase']]


In [4]:
selected_df[['Marital_Status', 'Product_Category_1', 'Purchase']].corr()

,Marital_Status,Product_Category_1,Purchase
Marital_Status,1.000000,0.019888,-0.000463
Product_Category_1,0.019888,1.000000,-0.343703
Purchase,-0.000463,-0.343703,1.000000


In [5]:
selected_df.head()


,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Purchase
0,F,0-17,10,A,2,0,3,8370
1,F,0-17,10,A,2,0,1,15200
2,F,0-17,10,A,2,0,12,1422
3,F,0-17,10,A,2,0,12,1057
4,M,55+,16,C,4+,0,8,7969


In [6]:
selected_df['Purchase']


0          8370
1         15200
2          1422
3          1057
4          7969
          ...  
550063      368
550064      371
550065      137
550066      365
550067      490
Name: Purchase, Length: 550068, dtype: int64

In [7]:
from sklearn.model_selection import train_test_split

X = selected_df.drop(columns=['Purchase'])
y = selected_df['Purchase']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train

,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1
178247,M,51-55,7,B,3,1,1
196647,M,18-25,12,C,4+,0,5
418590,M,26-35,5,B,1,0,8
408727,M,18-25,4,B,2,0,8
216416,M,18-25,4,B,1,0,1
...,...,...,...,...,...,...,...
110268,M,46-50,17,C,2,0,5
259178,M,26-35,14,C,2,1,5
365838,F,18-25,1,B,1,0,5
131932,F,18-25,4,B,2,0,8


In [8]:
X_train

,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1
178247,M,51-55,7,B,3,1,1
196647,M,18-25,12,C,4+,0,5
418590,M,26-35,5,B,1,0,8
408727,M,18-25,4,B,2,0,8
216416,M,18-25,4,B,1,0,1
...,...,...,...,...,...,...,...
110268,M,46-50,17,C,2,0,5
259178,M,26-35,14,C,2,1,5
365838,F,18-25,1,B,1,0,5
131932,F,18-25,4,B,2,0,8


In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
nominal_features = ['Gender', 'City_Category']
ordinal_features = ['Age', 'Stay_In_Current_City_Years']
numerical_columns = ['Occupation', 'Marital_Status', 'Product_Category_1']

In [10]:
df['Stay_In_Current_City_Years'].value_counts()


Stay_In_Current_City_Years
1     193821
2     101838
3      95285
4+     84726
0      74398
Name: count, dtype: int64

In [11]:
df['Gender'].value_counts()

Gender
M    414259
F    135809
Name: count, dtype: int64

In [12]:
preprocessor = ColumnTransformer(
    transformers=[
        ('ohe', OneHotEncoder(), nominal_features),
        ('le', OrdinalEncoder(), ordinal_features),
        ('ss', StandardScaler(), numerical_columns)
    ],
)

In [13]:

preprocessor


,transformers,"[('ohe', ...), ('le', ...), ...]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,categories,'auto'
,drop,None
,sparse_output,True


In [14]:
preprocessor.fit_transform(selected_df)[0]


array([ 1.        ,  0.        ,  1.        ,  0.        ,  0.        ,
        0.        ,  2.        ,  0.29486356, -0.83301799, -0.61080871])

In [15]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
#from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor())
])

In [16]:
pipeline.fit(X_train,y_train)

,steps,"[('preprocessor', ...), ('regressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('ohe', ...), ('le', ...), ...]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [17]:
X_test

,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1
84432,M,51-55,19,C,3,0,1
72724,F,36-45,11,B,1,0,1
197032,F,36-45,0,A,2,1,8
353704,M,18-25,4,A,3,0,1
91198,M,18-25,4,B,0,0,6
...,...,...,...,...,...,...,...
133324,M,55+,13,B,2,1,1
138718,F,26-35,4,B,3,0,8
22886,M,36-45,20,A,2,1,8
137110,M,36-45,7,A,0,0,11


In [ ]:
X_test['Purchase'] = y_test

In [19]:
X_test['Prediction'] = pipeline.predict(X_test)


In [26]:
X_test


,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Purchase,Prediction,Actual_Error,Absolute_Actual_Error,Absolute_Percentage_Error
84432,M,51-55,19,C,3,0,1,19142,16026.744141,3115.255859,3115.255859,0.162745
72724,F,36-45,11,B,1,0,1,15513,13530.408203,1982.591797,1982.591797,0.127802
197032,F,36-45,0,A,2,1,8,7802,7254.826172,547.173828,547.173828,0.070133
353704,M,18-25,4,A,3,0,1,15455,13057.036133,2397.963867,2397.963867,0.155158
91198,M,18-25,4,B,0,0,6,4492,15936.153320,-11444.153320,11444.153320,2.547674
...,...,...,...,...,...,...,...,...,...,...,...,...
133324,M,55+,13,B,2,1,1,19139,15145.119141,3993.880859,3993.880859,0.208678
138718,F,26-35,4,B,3,0,8,7856,7430.262207,425.737793,425.737793,0.054193
22886,M,36-45,20,A,2,1,8,4037,6781.355957,-2744.355957,2744.355957,0.679801
137110,M,36-45,7,A,0,0,11,7467,4518.012695,2948.987305,2948.987305,0.394936


In [ ]:
X_test['Actual_Error'] = X_test['Purchase'] - X_test['Prediction']
X_test['Absolute_Actual_Error'] = abs(X_test['Actual_Error'])
X_test['Absolute_Percentage_Error'] = X_test['Absolute_Actual_Error'] / X_test['Purchase']

In [22]:
X_test['Absolute_Percentage_Error'].mean()


np.float64(0.3725695569346725)

In [ ]:

import pickle
import pandas as pd

# Suppose 'pipeline' is your trained model/pipeline
with open("regrassion.pkl", "wb") as f:
    pickle.dump(pipeline, f)



df = pd.DataFrame(
    [['M', '51-55', 7, 'B', "3", 1, 1]],
    columns=[
        'Gender',
        'Age',
        'Occupation',
        'City_Category',
        'Stay_In_Current_City_Years',
        'Marital_Status',
        'Product_Category_1'
    ]
)

with open(r"F:\Data science\statistics\regrassion.pkl", "rb") as f:
    pipeline = pickle.load(f)

print(pipeline.predict(df))


[14315.327]
